In [4]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Cargar el CSV
df = pd.read_csv('citi.csv')

# Mostrar las primeras filas
print("Primeras 5 filas del dataset:")
display(df.head())

# Mostrar información general
print("\nInformación del dataset:")
print(df.info())

# Resumen estadístico
print("\nResumen estadístico:")
print(df.describe())

# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

In [ ]:
# Imputar valores nulos en columnas numéricas con la mediana
numeric_cols = ['TMAX', 'TMIN', 'PRCP', 'SNOW', 'AWND', 'trafico_volumen', 'rango_temperatura', 'promedio_ultimas_168_horas']
for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Verificar valores nulos nuevamente
print("Valores nulos después de imputación:")
print(df.isnull().sum())

# Eliminar filas con valores nulos en 'demanda' (si las hay)
df = df.dropna(subset=['demanda'])

# Confirmar limpieza
print("\nShape del dataset después de limpieza:", df.shape)

In [ ]:
# Codificar variables categóricas
categorical_cols = ['DIA_SEMANA', 'TIPO_BICICLETA', 'TIPO_USUARIO', 'START_STATION_ID']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Escalar variables numéricas
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Mostrar primeras filas después del preprocesamiento
print("Primeras 5 filas después de codificación y escalado:")
display(df.head())

In [ ]:
# Definir variables independientes (X) y dependiente (y)
X = df.drop(['demanda', 'FECHA', 'NOMBRE_ESTACION'], axis=1)
y = df['demanda']

# Dividir datos en entrenamiento y prueba (respetando orden temporal)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Mostrar dimensiones
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)

In [ ]:
# Crear datasets de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Definir parámetros de LightGBM
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Entrenar el modelo
model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[train_data, test_data])

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

In [ ]:
# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE en el conjunto de prueba: {rmse:.2f}")

# Visualizar predicciones vs. reales
plt.figure(figsize=(12, 6))
plt.plot(y_test.values[:100], label='Demanda Real', color='blue')
plt.plot(y_pred[:100], label='Demanda Predicha', color='red', linestyle='--')
plt.title('Demanda Real vs. Predicha (Primeras 100 observaciones)')
plt.xlabel('Observación')
plt.ylabel('Número de Viajes')
plt.legend()
plt.show()

In [ ]:
# Graficar importancia de variables
plt.figure(figsize=(10, 8))
lgb.plot_importance(model, max_num_features=10, importance_type='gain')
plt.title('Importancia de las Variables (Top 10)')
plt.show()

In [ ]:
# Guardar el modelo para uso futuro
model.save_model('lightgbm_bicicletas_model.txt')

# Cargar el modelo (ejemplo de cómo usarlo más tarde)
# loaded_model = lgb.Booster(model_file='lightgbm_bicicletas_model.txt')